In [2]:
import pandas as pd
import argparse
import util
from helpers import query_dscf, query_intake, clean_sample_id
import regex as re

In [ ]:
plog = pd.read_excel(util.print_log, sheet_name='LOG', header=0)


In [ ]:
plog.head()

In [ ]:
plog.columns

In [ ]:
pset = pd.DataFrame()
pset['temp'] = plog['CPT/EDTA brand, batch #'].dropna(axis=0).apply(lambda x: re.sub(r"[# ]","",str(x))).apply(lambda x: re.sub("Vacutainer","",str(x)))

pset['temp'] = pset['temp'].dropna(axis=0).apply(lambda x: re.sub("lastbag-openednewone-","",str(x))).apply(lambda x: re.sub("openednewCPTstartingforC3206,","",str(x)))

pset['temp'] = pset['temp'].dropna(axis=0).apply(lambda x: re.sub("D,3","D3",str(x))).apply(lambda x: re.sub(",",";",str(x)))

pset['temp'] = pset['temp'].dropna(axis=0).apply(lambda x: re.split(";",x))


In [ ]:
catalog = []
lot = []
lot_dup = {}
for n, list in enumerate(pset['temp']):
    # if len(list) >1:
    for item in list:
        if "bd" in item or "BD" in item:
            catalog.append(item)
        elif len(lot) == n:
            lot_dup.update({n:item})
            continue
        elif "Lot" in item or len(item) == 7 and "/" not in item:
            lot.append(item)
    if len(catalog) != n+1:
        catalog.append("No Catalog Found")

    if len(lot) != n-1:
        print(len(lot), n)
        lot.append("No Lot Found")

pset['catalog'] = catalog
pset['lot'] = lot

In [ ]:
pset